In [75]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV


In [46]:
df = pd.read_csv('data/weatherAUS.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [47]:
df.isnull().sum().sum()

343248

In [48]:
round (df['Sunshine'].isnull().sum() * 100 / 145460)

48

In [49]:
for column in df.columns:
    if df[column].isnull().sum() * 100 / 145460 > 40:
        df = df.drop([column], axis=1)
    else:
        continue

In [50]:
def category (x):
    if x == 'No':
        return 0
    elif x == 'Yes':
        return 1
    else:
        return x

In [51]:
df['RainToday'] = df['RainToday'].apply(category)
df['RainTomorrow'] = df['RainTomorrow'].apply(category)

In [52]:
round(df['RainToday'].mean(), 2)

0.22

In [53]:
df['Month'] = pd.to_datetime(df['Date']).dt.month

In [54]:
df = df.drop('Date', axis=1)

In [55]:
categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']

In [56]:
df_dummies = pd.get_dummies(df, columns=categoricals)

In [57]:
df_dummies

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,...,0,0,0,0,0,0,0,0,1,0
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,...,0,0,0,0,0,0,0,0,0,1
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,...,0,0,0,0,0,0,0,0,0,1
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,...,0,0,0,0,0,0,0,0,0,0
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2.8,23.4,0.0,31.0,13.0,11.0,51.0,24.0,1024.6,1020.3,...,0,0,0,0,0,0,0,0,0,0
145456,3.6,25.3,0.0,22.0,13.0,9.0,56.0,21.0,1023.5,1019.1,...,0,0,0,0,0,0,0,0,0,0
145457,5.4,26.9,0.0,37.0,9.0,9.0,53.0,24.0,1021.0,1016.8,...,0,0,0,0,0,0,0,0,1,0
145458,7.8,27.0,0.0,28.0,13.0,7.0,51.0,24.0,1019.4,1016.5,...,0,0,0,0,0,0,0,0,0,0


In [58]:
df_dummies = df_dummies.dropna(axis=0)

In [59]:
df_dummies.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,...,0,0,0,0,0,0,0,0,1,0
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,...,0,1,0,0,0,0,0,0,0,0
6,14.3,25.0,0.0,50.0,20.0,24.0,49.0,19.0,1009.6,1008.2,...,0,0,0,0,0,0,0,1,0,0
11,15.9,21.7,2.2,31.0,15.0,13.0,89.0,91.0,1010.5,1004.2,...,0,0,0,0,0,0,0,0,0,0
12,15.9,18.6,15.6,61.0,28.0,28.0,76.0,93.0,994.3,993.0,...,1,0,0,0,0,0,0,0,0,0


In [60]:
X = df_dummies.drop('RainTomorrow', axis=1)
y = df_dummies['RainTomorrow']

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=31)

In [62]:
y_test.mean()

0.22770253002811142

In [63]:
def gbs(data, n):     
    inds = np.random.randint(0, len(data), (n, len(data))) #определяем индексы случайным образом
    numbers = data[inds] #выбираем значения по индексам
    return numbers
target = X_train['MinTemp'].values #выбираем целевую переменную
np.random.seed(31) #задаём параметр генератора случайных чисел
mean_values = [np.mean(x) for x in gbs(target, 1000)] #получаем все средние значения
np.std(mean_values) #находим для них стандартное отклонение

0.02879072820657669

In [64]:
log_reg = linear_model.LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred_train1=log_reg.predict(X_train)
y_pred_test1=log_reg.predict(X_test)

print (f'ROC-AUC на обучающей выборке {metrics.roc_auc_score(y_train, y_pred_train1):.3f}')
print (f'ROC-AUC на тестовой выборке {metrics.roc_auc_score(y_test, y_pred_test1):.3f}')

ROC-AUC на обучающей выборке 0.728
ROC-AUC на тестовой выборке 0.727


c:\Users\kuzin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [65]:
dec_tree = DecisionTreeClassifier(random_state=42)

In [66]:
#GridSearchCV
params = {'max_leaf_nodes': list(range(2, 10)), 
          'min_samples_split': [2, 3, 4], 
          'max_depth': [5,7,9,11]}

grid_search_cv = GridSearchCV( estimator= dec_tree, 
                              param_grid = params,
                              scoring = metrics.roc_auc_score,
                              cv = 3)

In [67]:
grid_search_cv.fit(X_train, y_train)

c:\Users\kuzin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\kuzin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: roc_auc_score() takes 2 positional arguments but 3 were given

  warnings.warn(
c:\Users\kuzin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\kuzin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [5, 7, 9, 11],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9],
                         'min_samples_split': [2, 3, 4]},
             scoring=<function roc_auc_score at 0x000001A0FD723C10>)

In [68]:
print (grid_search_cv.best_params_)

{'max_depth': 5, 'max_leaf_nodes': 2, 'min_samples_split': 2}


In [71]:
clf = DecisionTreeClassifier(max_depth=5, max_leaf_nodes=9, min_samples_split=2, random_state=42)

In [72]:
clf.fit(X_train, y_train)
y_pred_train2=clf.predict(X_train)
y_pred_test2=clf.predict(X_test)
print (f'ROC-AUC на тестовой выборке {metrics.roc_auc_score(y_test, y_pred_test2):.2f}')

ROC-AUC на тестовой выборке 0.70


In [76]:
rand_forest=RandomForestClassifier(n_estimators=100, random_state=31)
rand_forest.fit(X_train, y_train)
y_pred_train3=rand_forest.predict(X_train)
y_pred_test3=rand_forest.predict(X_test)

print (print (f'ROC-AUC на тестовой выборке {metrics.roc_auc_score(y_test, y_pred_test3):.2f}'))

ROC-AUC на тестовой выборке 0.73
None


In [80]:
params1 = {'max_features': [ 4, 5, 6, 7], 'min_samples_leaf': [3, 5, 7, 9, 11], 'max_depth': [5, 10, 15]}
grid_search_cv1=GridSearchCV(estimator = RandomForestClassifier(random_state=31), param_grid= params1, cv=3)

In [81]:
grid_search_cv1.fit(X_train, y_train)

print (grid_search_cv1.best_params_)

{'max_depth': 15, 'max_features': 7, 'min_samples_leaf': 3}


In [84]:
best_model = RandomForestClassifier(n_estimators=100, max_depth=15, max_features=7, min_samples_leaf=3, random_state=31)

best_model.fit(X_train, y_train)

y_pred_train4=best_model.predict(X_train)
y_pred_test4=best_model.predict(X_test)

print (print (f'ROC-AUC на тестовой выборке {metrics.roc_auc_score(y_test, y_pred_test4):.2f}'))

ROC-AUC на тестовой выборке 0.70
None


In [86]:
feature_names = [x for x in df_dummies if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': best_model.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.250783
2,Rainfall,0.079757
6,Humidity9am,0.070403
10,Cloud9am,0.067092
9,Pressure3pm,0.065272
...,...,...
50,Location_Newcastle,0.000000
62,Location_SalmonGums,0.000000
51,Location_Nhil,0.000000
52,Location_NorahHead,0.000000
